In [ ]:
# Criado por Alessadra Ulinick

# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

# importar biblioteca pandas e numpy
import pandas as pd
import numpy as np
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import functions as sf
import time

In [ ]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
# 1 - Carregar DataFrame do PySpark
data = sc.read.csv("mock_data_1.csv", inferSchema=True, header=True, sep=',')

# Mostrar o arquivo
data

In [ ]:
# Convertendo Spark para pandas
dataframe = data.toPandas()

In [ ]:
# 2 - unir "nome" e "sobrenome" em uma nova coluna chamada "nome_completo"
dataframe = pd.DataFrame(dataframe,columns=["id","nome","sobrenome", "email", "salario", "ultima_compra"]) 
dataframe["nome_completo"] = dataframe["nome"] + " " + dataframe["sobrenome"]

In [ ]:
# 3 - Remover lihas com valores nulos da coluna "ultima_compra"
dataframe = dataframe.dropna(subset=["ultima_compra"])

# Saber quantos valores estão com NaN = 104 valores NaN antes de remove-los
dataframe.isnull().sum()

In [ ]:
# 4 - Remover substring "$" da coluna "salario"
dataframe["salario"] = dataframe["salario"].str.replace("$", "")

In [ ]:
# 5 - Alteração do tipo de dado
dataframe["salario"] = dataframe["salario"].astype(float)

In [ ]:
# 6 - Conversão para timestamp
dataframe["ultima_compra"] = pd.to_datetime(dataframe["ultima_compra"],format= '%H:%M:%S' ).dt.time

In [ ]:
# 7 - Salário médio dos clientes
dataframe["salario"].mean()